In [2]:
# import libraries

import requests 
import pandas as pd 

In [126]:
#set API Key (it changes every 24h!)

API_key = 'RGAPI-2c656568-80f9-431f-bc10-5516b90facaa'

In [50]:
# set the URL for D1 in KR

urls = ['https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?', 'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/II?']

# initializing all the pages we want to go through 

pages = ["I","II","III","IV","V","VI","VII","VIII","IX","X","XI","XII","XIII",'XIV','XV']

In [40]:
# let's create a function that will return us all summoner ids from the leagues below masters

def get_summoner_ids(pages, url):

    json_list = []

    for i in range(len(pages)):
        response = requests.get(f"{url}page={i}&api_key={API_key}").json()
        json_list.append(response)
    
    player_ids = []

    for i in range(1,len(pages)):
        for j in range(len(json_list[i])):
            if json_list[i][j].get('inactive') == False:
                player_ids.append(json_list[i][j]['summonerId'])
                
    return player_ids

In [51]:
# nowe let's actually get all the ids for players in diamond

player_ids_diamond = []

for url in urls:
    player_ids_diamond.append(get_summoner_ids(pages,url))

In [70]:
# now let's do the same for master players

url_masters = "https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?"

def get_master_ids(url):
    
    response = requests.get(f"{url}api_key={API_key}").json()
    player_ids = []
    for j in range(len(response['entries'])):
        if response['entries'][j].get('inactive') == False:
                player_ids.append(response['entries'][j]['summonerId'])
    return player_ids

master_ids = get_master_ids(url_masters)

In [73]:
# we also need grandmaster ids 

url_grandmasters = "https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?"

grandmaster_ids = get_master_ids(url_grandmasters)


In [77]:
# finally I want challenger ids 

url_challenger = "https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?"

challenger_ids = get_master_ids(url_challenger)

In [119]:
# we have summoner ids, but we actually need puuids to be able to get matchids. We can only get puuids with summonerids, that why we needed them
import time

def get_puuids(ids, max_retries=5, retry_delay=10):
    puuid_ids = []
    url = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/"
    for i in range(len(ids)):
        for retry_count in range(max_retries):
            try:
                response = requests.get(f"{url}{ids[i]}?api_key={API_key}").json()
                puuid_ids.append([ids[i], response["puuid"]])
                break  # Exit the retry loop if we got a successful response
            except requests.exceptions.RequestException as e:
                print(f"Error occurred while fetching puuid for ID {ids[i]}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                if retry_count == max_retries - 1:
                    # If we've exhausted all retries, return the puuid_ids we have collected so far
                    print(f"Failed to fetch puuid for ID {ids[i]}. Moving on...")
                    return puuid_ids
            except Exception as e:
                print(f"An unexpected error occurred while fetching puuid for ID {ids[i]}. Moving on...")
                return puuid_ids
    return puuid_ids

In [ ]:
# getting challenger, grandmaster and master puuids 

challenger_puuid = get_puuids(challenger_ids)
grandmaster_puuid = get_puuids(grandmaster_ids)
master_puuid = get_puuids(master_ids)

In [120]:
# i still need diamonds puuids

diamond_puuid = []

for i in range(len(player_ids_diamond)):
    diamond_puuid.append(get_puuids(player_ids_diamond[i]))

# only take unique match ids into the list 
# then from each match id i need to take a list of champions + items that each of those champions built 

Error occurred while fetching puuid for ID jeaz-jPScgIQycvHe6Pm-i_kMmdWc9_wPFZzHequkWr0rtfo. Retrying in 10 seconds...


In [122]:
# writing a function that will get me all puids to one array
def concat_puuids(d,m,gm,ch):
    all_puids = []
    
    for id in m:
        all_puids.append(id)
    for id in gm:
        all_puids.append(id)
    for id in ch:
        all_puids.append(id)
    for i in range(len(d)):
        for j in range(len(d[i])):
            all_puids.append(d[i][j][1])

    return all_puids

In [123]:
# getting all puids together

all_puids = concat_puuids(diamond_puuid,master_puuid,grandmaster_puuid,challenger_puuid)


# so far we have 12000 active players, that looks promising - we should be getting about 30 matches on average from each of them which even 
# in a situation where the matches will double it will still give us a lot of unique matches


In [3]:
# let's export the puuids for the future so we dont have to run the code again with every patch
import json

def export_to_json(arr, filename):
    with open(f'{filename}.json','w') as f:
        json.dump(arr, f)



In [ ]:
export_to_json(all_puids, "puuids_latest_patch")

In [2]:
# function to load the puuids

import json

def load_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    return data


In [136]:
# now i need to find match ids for all the puuids 


def get_match_ids(puuids,max_retries=5, retry_delay=10):

    last_patch_date = '1676899200'
    match_ids =[]
    url = "https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/"

    for id in puuids:
        for retry_count in range(max_retries):
            try:
                response = requests.get(f"{url}{id}/ids?startTime={last_patch_date}&start=0&count=30&api_key={API_key}").json()
                match_ids.append(response)
                break
            except requests.exceptions.RequestException as e:
                print(f"Error occurred while fetching puuid for ID {id}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                if retry_count == max_retries - 1:
                    # If we've exhausted all retries, return the puuid_ids we have collected so far
                    print(f"Failed to fetch puuid for ID {id}. Moving on...")
                    return match_ids
            except Exception as e:
                print(f"An unexpected error occurred while fetching puuid for ID {id}. Moving on...")
                return match_ids
    return match_ids

In [137]:
# get match ids

all_match_ids = get_match_ids(all_puids)

Error occurred while fetching puuid for ID XTsa2yO1q3r2VFzvDnyZ6tIwoXrF08IEIodfzgfg46iHytB4hDTjpQlVvb2ycK5JgAhxk5gDrMEnNw. Retrying in 10 seconds...
Error occurred while fetching puuid for ID Sa6hAq_4dEo8o0jDpw52BYgiVpRg8KNTXIE7okETrsnqSFSE44nJ6Fq4fFcXBmnkjhgnp16p4q8dOA. Retrying in 10 seconds...
Error occurred while fetching puuid for ID ZM7HxuCE9dL09js3Yv8aCwv-qOptUpfCesmWEVRZwiKn7Dbzs2Pl8BhaM2bkKXuXDDl9AXbv2j_bqg. Retrying in 10 seconds...


In [ ]:
# ok, we have almost 200k unique match ids, looking good - we will have a lot of data
# first let's load the match ids into json file so we don't lose them

export_to_json(unique_match_ids.tolist(), 'match_ids_latest_patch')

# let's load the json file with items 

items = load_json('C:/Users/kaczm/OneDrive/Dokumenty/item.json')

In [4]:
# let's eliminate duplicates

import numpy as np

def get_pure_match_ids(match_ids):
    pure_match_ids = []

    for i in range(len(match_ids)):
        pure_match_ids.append(match_ids[i])
            
    return np.unique(pure_match_ids) 
    
unique_match_ids = get_pure_match_ids(load_json('match_ids_latest_patch.json'))  

In [9]:
# let's get all the items name 

def get_items(item_json):
    dic = {}
    for key, value in item_json['data'].items():
        dic[key] = value['name']
    return dic

item_data = get_items('C:/Users/kaczm/OneDrive/Dokumenty/item.json')

TypeError: string indices must be integers

In [8]:
# let's also get rune data

def get_runes(rune_json):

    dic = {}
    with open(rune_json) as f:
        data = json.load(f)

    for rune in data:
        for slot in rune['slots']:
            for sub_rune in slot['runes']:
                dic[sub_rune['id']] =  sub_rune['name']
    return dic

rune_data = get_runes('C:/Users/kaczm/OneDrive/Dokumenty/runesReforged.json')

In [5]:
# now we need the data coming from match_ids
# but what do we need, aside from champions played, runes, stats and who won? maybe the blue/red side played? summoner spells i dont think are needed

# we will need multiple threads to quicken the execution of the function, as 195k elements would actually get called over 18 days 

import concurrent.futures
import requests
import time

def get_match_data(match_ids, max_retries=5, retry_delay=10, num_threads=1):

    participants_data = []

    def fetch_match_data(match_id):
        for retry_count in range(max_retries):
            try:

                response = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={API_key}').json()

                for participant in response['info']['participants']:
                    participant_data = {
                        'kills': participant['kills'],
                        'deaths': participant['deaths'],
                        'assists': participant['assists'],
                        'champion_name': participant['championName'],
                        'team_position': participant['teamPosition'],
                        'items': [participant['item0'], participant['item1'], participant['item2'], participant['item3'], participant['item4'], participant['item5'], participant['item6']],
                        'win': participant['win'],
                        'masteries': {
                            'primary_master': participant['perks']['styles'][0]['selections'][0]['perk'],
                            'primary_master_sub_1': participant['perks']['styles'][0]['selections'][1]['perk'],
                            'primary_master_sub_2': participant['perks']['styles'][0]['selections'][2]['perk'],
                            'primary_master_sub_3': participant['perks']['styles'][0]['selections'][3]['perk'],
                            'secondary_mastery': participant['perks']['styles'][1]['selections'][0]['perk'],
                            'secondary_mastery_sub_1': participant['perks']['styles'][1]['selections'][1]['perk']
                        }
                    }
                    participants_data.append(participant_data)
                break 
            except requests.exceptions.RequestException as e:
                print(f"Error occurred while fetching match ids for ID {match_id}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                if retry_count == max_retries - 1:
                    # If we've exhausted all retries, return None
                    print(f"Failed to fetch match id for ID {match_id}. Moving on...")
                    return None
            except Exception as e:
                print(f"An unexpected error occurred while fetching match id for ID {match_id}. Moving on...")
                return participants_data

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_match_id = {executor.submit(fetch_match_data, match_id): match_id for match_id in match_ids}

        for future in concurrent.futures.as_completed(future_to_match_id):
            match_id = future_to_match_id[future]
            participant_data = future.result()
            if participant_data is not None:
                participants_data.append(participant_data)

    return participants_data
    

In [127]:
players_data_batch_one = get_match_data(unique_match_ids[170000:190000])

Error occurred while fetching match ids for ID KR_6418184376. Retrying in 10 seconds...
Error occurred while fetching match ids for ID KR_6418329042. Retrying in 10 seconds...
Error occurred while fetching match ids for ID KR_6418390951. Retrying in 10 seconds...
Error occurred while fetching match ids for ID KR_6418411271. Retrying in 10 seconds...
Error occurred while fetching match ids for ID KR_6418413563. Retrying in 10 seconds...
Error occurred while fetching match ids for ID KR_6418430462. Retrying in 10 seconds...
Error occurred while fetching match ids for ID KR_6418433221. Retrying in 10 seconds...
Error occurred while fetching match ids for ID KR_6418452216. Retrying in 10 seconds...
Error occurred while fetching match ids for ID KR_6418509885. Retrying in 10 seconds...
Error occurred while fetching match ids for ID KR_6418530603. Retrying in 10 seconds...


In [51]:
len(players_data_batch_one)

199991

In [124]:
len(players_data_batch_one[137701:])

62290

In [52]:
list(unique_match_ids).index('KR_6417436762')

163766

In [125]:
export_to_json(players_data_batch_one[137701:], 'players_match_data_thirteen')